## Naive Bayes on Political Text

### Sindhu Bhattarai

### Assignment 4.1

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import pandas as pd
import nltk
import random
import re
import emoji
import numpy as np
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation


import os

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
# Place any addtional functions or constants you need here. 

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison

# Stopwords
sw = stopwords.words("english")


def common_tokens(tokens):
    return (Counter(tokens).most_common())

    
# def is_emoji(s):
#     return(emoji.is_emoji(s))

# def contains_emoji(s):    
#     s = str(s)
#     emojis = [ch for ch in s if is_emoji(ch)]

#     return(len(emojis) > 0)


def remove_stop(text, stop_words = sw) :
     # modify this function to remove stopwords
    return(" ".join([word for word in text.split(' ') if word not in stop_words ]))


#changes as per the assignment requirement
punctuation.add('’')
punctuation.add('" "')
tw_punct = punctuation #- {"#"}

def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    # modified this function to return tokens of word only 
    return text.split(" ")

def lower_case(text):
    return text.casefold()

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)



In [3]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes. 

In [4]:
stored_name = convention_cur.execute("SELECT name FROM sqlite_master")
stored_name.fetchone()

('conventions',)

In [5]:
my_pipeline = [str.lower, remove_punctuation, remove_stop,tokenize]

In [6]:

query_results = convention_cur.execute(
                            '''
                            SELECT *  
                            FROM conventions
                            ''')



In [7]:
#checking the rows of stored value 
# for row in query_results :
#     print(row[5])
    #print(row[0])



In [8]:

convention_data = []
# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 



for row in query_results :
    text = row[5]
    # Thefirst element in the sublist should be the cleaned and tokenized(using pipeline)
    clean_text = prepare(text, pipeline = my_pipeline) 
    #tokenized text should be in a single string 
    string_text = " ".join(clean_text)
    #The second element should be the party.
    party = row[0] 
    data_to_append = [string_text, party] 
    convention_data.append(data_to_append)



In [9]:
#convention_data

Let's look at some random entries and see if they look right. 

In [10]:
random.choices(convention_data,k=10)

[['young people watching dont give america despite flaws problems weve come far still equal country opportunities previous generations could never imagined theres lot heartbreak america truth many things broken pandemic saying goes world breaks everyone afterward many strong broken places thats joe biden knows keep going unify lead hes done family country come november strong together well heal together well redeem soul',
  'Democratic'],
 ['built beautiful family circle trust', 'Democratic'],
 ['lets clear vaccine racism got work george floyd brianna taylor lives many others name children us weve got work fulfill promise equal justice law heres thing none us free us free',
  'Democratic'],
 ['obama biden led united nations denounce friend ally israel president trump moved embassy jerusalem un tried condemn us proud cast american veto president record strength success former vice president record weakness failure joe biden good iran isis great communist china hes godsend everyone wants

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [11]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]
# tokens = [i for i in tokens if not i in sw] 

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2387 as features in the model.


In [12]:
#feature_words

In [13]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    
    ret_dict = dict()
    token = text.lower().split()
    for word in token:
        if word in fw:
            ret_dict[word] = True
        
    return(ret_dict)

In [14]:
conv_features('QuiCk QuiCk brOwn foX',{'quick','fox','jumps'})

{'quick': True, 'fox': True}

In [15]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [16]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]
#featuresets


In [17]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [18]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.498


In [19]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

I can see that among top 25 token which are most informative features by classifier most of the tokens are more often used by republican with very high ratio in comparision.
The tokens "climate" and "votes" are mostly used by democratic party. As we know democrats talked about climate change in heafty amount in 2020 election which teh classifier definitely addressess. 

We can see that the token such as "enforcement", "china","defense", "crime", "trade", "destroy", "freedoms","drug","isis", "wonderful" are more are heavily used by republicans more than democrats. These words undoubtedly represents the sentiments of rebublicans and their representation. 

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [20]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [21]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [22]:
#checking the rows of stored value 
#for row in results:
#     print(row[1])
    #print(row[2])


In [23]:
tweet_data = []
# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

for row in results :
    #tweets is in row[2] and decoding b prefix from the data
    tweets = row[2].decode('utf-8')
   # Thefirst element in the sublist should be the cleaned and tokenized(using pipeline)
    clean_tweets = prepare(tweets, pipeline = my_pipeline) 
    #tokenized text should be in a single string 
    string_tweets = " ".join(clean_tweets)
    #The second element should be the party.
    party = row[1] 
    tweetdata_to_append = [string_tweets, party] 
    tweet_data.append(tweetdata_to_append)


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [24]:
random.seed(20201014)
tweet_data_sample = random.choices(tweet_data,k=10)
tweet_data_sample

[['earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv',
  'Democratic'],
 ['go tribe rallytogether httpstco0nxutfl9l5', 'Democratic'],
 ['apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh',
  'Democratic'],
 ['grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line\n\nhttpstcoezpv0vmiz3',
  'Republican'],
 ['lets make even greater  kag 🇺🇸 httpstcoy9qozd5l2z', 'Republican'],
 ['1hr cavs tie series 22 im allin216 repbarbaralee scared roadtovictory',
  'Democratic'],
 ['congrats belliottsd new gig sd city hall glad continue serve… httpstcofkvmw3cqdi',
  'Democratic'],
 ['really close 3500 raised toward match right whoot thats 7000 nonmath majors room 😂 help us get httpstcotu34c472sd httpstcoqsdqkypsmc',
  'Democratic'],
 ['today comment period potuss plan

In [25]:

for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet,feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line

httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: lets make even greater  kag 🇺🇸 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1hr cavs t

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [26]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet,feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [27]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3724, 'Democratic': 554}),
             'Democratic': defaultdict(int,
                         {'Republican': 4834, 'Democratic': 890})})

### Reflections

We can see in the above results that for overall republican tweets  3724 classified as rebublican and 554 classified as democratic. 
Similarly, out of Democratic tweets 4834 classified as republican and 890 as democratic. 

Our classifier is classifying rebublican more accurately than democratic. It falsely classifies democratic as republican in heavy number. Overall we can say that the classifier is not doing a good job here. There can be various reason which can lead to classifier not working accurately. Some of them are:

1. There can be class imbalance. More unique tokens of rebublican than of democratic party.
2. Some other classifier might work better than naive bayes for our data. Here the classifier is overfitting for republican class and underfitting for democratic class. 


